In [1]:
import import_ipynb
import numpy as np
import pandas as pd
from data_files import elig
from patient_visits import PATIENT_VISITS, missed_sfs, fill_status, log_461_014
from central_functions import read_files, format_study_id, lowercase_all_vars, insert_row, remove_nan_cols

importing Jupyter notebook from data_files.ipynb
importing Jupyter notebook from central_functions.ipynb
C:/Users/dgao/Box/Intern Data Models/461-012/dta_demographics.txt
C:/Users/dgao/Box/Intern Data Models/461-012/dta_iex_proc.txt
C:/Users/dgao/Box/Intern Data Models/461-012/dta_sbj_participation.txt
C:/Users/dgao/Box/Intern Data Models/461-012/dta_subject_termination.txt
C:/Users/dgao/Box/Intern Data Models/461-012/dta_time_table.txt
C:/Users/dgao/Box/Intern Data Models/461-012/elig_enroll_log.pdf
C:/Users/dgao/Box/Intern Data Models/461-012/myk461-012_visit.csv
C:/Users/dgao/Box/Intern Data Models/461-012/udp_pi_eligibility_statement.txt
importing Jupyter notebook from patient_visits.ipynb


/// Cleaning ELIGIBILITY data ///

These eligibility data come from the Rave MediData EDC system

These data represent high level data on each patient's study eligibility and the reason(s) why they were not eligible; each reason for ineligibility is represented by a separate row, therefore there may be multiple rows per one patient.

For screen_fails table, original variables include:
(DROPPED) projectid, studyid, environmentName, subjectId, StudySiteId, SiteGroup, instanceId, InstanceName, InstanceRepeatNumber, folderid, Folder, FolderName, FolderSeq, TargetDays, DataPageId, DataPageName, PageRepeatNumber, RecordDate, RecordId, RecordPosition, MinCreated, MaxUpdated, SaveTS, CODER_HIERARCHY, StudyEnvSiteNumber
project: study ID in the form of MYK######
Subject: patient ID ###-###-###-###
Site: text string name of site
SiteNumber: site ID number
TIVERS: "What version of the protocol was the subject enrolled"
IEYN_CV,IEYN_STD: "Did the subject meet all eligibility criteria?"
ECADAT: "Eligibility criteria assessment date"
IECAT: Inclusion or exclusion criteria not met
TIVERS: Version of protocol patient consented to
IETESTC: Inclusion/Exclusion identifier for crtierion not met
IERECO: "Has the subject reconsented to a different version of the protocol?"
IEHCM2_CV: "Did subject enroll within 28 days of MAVERICK HCM Week 24 visit?"
IECHRT: Cohort (MAVERICK or EXPLORER)
IERSCRN_CV: Subject approved for rescreen
(DROPPED) IEHCM_CV: "Did subject enroll within 28 days of MAVERICK-HCMWeek 24 visit?"
(DROPPED) IECMRPAR: "Subject participating in CMR sub study?"
(DROPPED) IERECO: "Subject reconsented to a different version of theProtocol?"

In [2]:
elig
for i in range(len(elig)):
    print(i,elig[i])

0 C:/Users/dgao/Box/Intern Data Models/224-001/224_001_elig_enroll_log.csv
1 C:/Users/dgao/Box/Intern Data Models/224-001/elig_enrollment_log.pdf
2 C:/Users/dgao/Box/Intern Data Models/461-005/elig.sas7bdat
3 C:/Users/dgao/Box/Intern Data Models/461-005/eligibility.sas7bdat
4 C:/Users/dgao/Box/Intern Data Models/461-006/elig.sas7bdat
5 C:/Users/dgao/Box/Intern Data Models/461-006/myk461006_eligibility_20200103164652.csv
6 C:/Users/dgao/Box/Intern Data Models/461-007/blinded_myk461007_eligibility_20200616130001.txt
7 C:/Users/dgao/Box/Intern Data Models/461-007/elig.sas7bdat
8 C:/Users/dgao/Box/Intern Data Models/461-008/elig.sas7bdat
9 C:/Users/dgao/Box/Intern Data Models/461-008/myk461008_eligibility_20200604010000.txt
10 C:/Users/dgao/Box/Intern Data Models/461-012/dta_iex_proc.txt
11 C:/Users/dgao/Box/Intern Data Models/461-012/elig_enroll_log.pdf
12 C:/Users/dgao/Box/Intern Data Models/461-013/elig_enroll_counts.xls
13 C:/Users/dgao/Box/Intern Data Models/461-014/dta_iex_proc.txt
1

In [3]:
# Original raw data eligibility files
e_224_001 = read_files(elig[0])
e1_461_005 = read_files(elig[2])
e1_461_006 = read_files(elig[4])
e2_461_007 = read_files(elig[7])
e1_461_008 = read_files(elig[8])
# Seems to only include randomized patients
e1_461_012 = read_files(elig[10])
# This is the elig enrollment log PDF file, which contains SID for each subject, which may
# correspond to id_participation from the previous dta_iex file, but NONE of the IDs match
# from one to the other; 'New Patient' column may indicate unique patients vs. rescreened
e2_461_012 = read_files(elig[11])[0].iloc[:42,:][['SID','Media']]
# e_461_013 = read_files(elig[12]) -- not usable
e1_461_015 = read_files(elig[15],skiprows=8).iloc[:55,:]
e1_461_018 = read_files(elig[16])
e2_461_018 = read_files(elig[17])

In [4]:
e2_012 = set(e2_461_012.SID.values)
e1_012 = set(e1_461_012.id_subject.values)
e1_461_012 = pd.merge(e1_461_012,e2_461_012,how='left',left_on='id_subject',right_on='SID').drop(['SID'],axis=1).rename(columns={'Media':'recruit_type'})
# e1_012 & e2_012

In [5]:
for i in range(len(elig)):
    print(i,elig[i])

0 C:/Users/dgao/Box/Intern Data Models/224-001/224_001_elig_enroll_log.csv
1 C:/Users/dgao/Box/Intern Data Models/224-001/elig_enrollment_log.pdf
2 C:/Users/dgao/Box/Intern Data Models/461-005/elig.sas7bdat
3 C:/Users/dgao/Box/Intern Data Models/461-005/eligibility.sas7bdat
4 C:/Users/dgao/Box/Intern Data Models/461-006/elig.sas7bdat
5 C:/Users/dgao/Box/Intern Data Models/461-006/myk461006_eligibility_20200103164652.csv
6 C:/Users/dgao/Box/Intern Data Models/461-007/blinded_myk461007_eligibility_20200616130001.txt
7 C:/Users/dgao/Box/Intern Data Models/461-007/elig.sas7bdat
8 C:/Users/dgao/Box/Intern Data Models/461-008/elig.sas7bdat
9 C:/Users/dgao/Box/Intern Data Models/461-008/myk461008_eligibility_20200604010000.txt
10 C:/Users/dgao/Box/Intern Data Models/461-012/dta_iex_proc.txt
11 C:/Users/dgao/Box/Intern Data Models/461-012/elig_enroll_log.pdf
12 C:/Users/dgao/Box/Intern Data Models/461-013/elig_enroll_counts.xls
13 C:/Users/dgao/Box/Intern Data Models/461-014/dta_iex_proc.txt
1

In [6]:
all_data = [e_224_001,e1_461_005,e1_461_006,e2_461_007,e1_461_008,e1_461_012,e1_461_015,e1_461_018,e2_461_018]

In [7]:
def rename_cols_for_sf(df):
    df = df.rename(columns={'actual_dt':'assess_date','subject id':'patient_id','screening_dt':'assess_date','sponsor_name':'in_ex_code','date of informed consent':'assess_date','id_participation':'patient_id','study_no_sponsor':'study_id','project':'study_id','subject':'patient_id','sitenumber':'site_id','tivers_cv':'protocol_num','ecadat':'assess_date','iecat':'in_or_ex','ietestc':'in_ex_code','inc_exc':'in_or_ex'},errors='ignore')
    return df

In [8]:
def subset_sf_for_sf(df):
    sf = df[(df.ieyn == 'N') | (df.ieyn == 'No')]
    sf = sf.drop(['ieyn','ieyn_cv','ieyn_std'],axis=1,errors='ignore')
    return sf

In [9]:
def create_new_in_ex_cols(df):
    df = df.rename(columns={'in_ex_code':'in_id'})
    df['in_id'] = df.in_id
    df['ex_id'] = 0.0
    return df

In [10]:
def merge_in_ex_codes(groupby_df):
    inc = ''
    exc = ''
    for i,row in groupby_df.iterrows():    
        if row.in_or_ex == 'IN':
            inc += str(row.in_ex_code)+','
        if row.in_or_ex == 'EX':
            exc += str(row.in_ex_code)+','
    groupby_df['in_id'] = inc
    groupby_df['ex_id'] = exc
    groupby_df['in_id'] = groupby_df.in_id.fillna('')
    groupby_df['ex_id'] = groupby_df.ex_id.fillna('')
    return groupby_df

In [11]:
def format_in_ex_code(in_ex_code):
    if pd.isnull(in_ex_code):
        in_ex_code = 'No Data'
        return in_ex_code
    if type(in_ex_code) == str:
        in_ex_code = int(in_ex_code[-2:])
    return in_ex_code

In [12]:
def drop_cols_for_sf(df):
    df = df.drop(['screening status','id_cb_crit','iehcm2_cv','randomisation no.','id_subject','cb_crit_value_text','protocol_pending','cb_crit_decision','period_no','valid','trialday_name','ie_population','trialday_no','runin_no','sbj_sex','cb_crit_deviation_cmt','cb_crit_free_na_cmt','cb_crit_pending','cb_crit_fulfilled','cb_ref_procedure','setup_version','control_no','continuous_name','continuous_value','period_kind','study_name','period_name','measurept_name','procedure_state_name','tolerance_pre','id_rsl_procedure','tolerance_post','procedure_name','siteid','studyid', 'ierscrn', 'iehcm_cv', 'ecadat_int', 'iechrt_cv', 'iehcm', 'iehcm2',
 'ietestc_cv','ietestc_raw', 'iereco_cv', 'iereco', 'dsdat_raw', 'dsdat_c', 'dsdat', 'dsdat_int', 'tivers', 'iecat_cv',
 'iecat_std','projectid', 'environmentname', 'subjectid', 'instanceid', 'instancename', 'instancerepeatnumber', 'folderid',
 'folder','foldername', 'folderseq', 'targetdays', 'datapageid', 'datapagename', 'pagerepeatnumber', 'recorddate', 'recordid',
 'dsdat_y', 'dsdat_yyyy', 'dsdat_m', 'dsdat_mm', 'dsdat_dd', 'dsdat_d', 'ecadat_c', 'ecadat_raw', 'ecadat_y', 'ecadat_yyyy',
 'ecadat_m', 'ecadat_mm', 'ecadat_d', 'ecadat_dd', 'recordposition', 'coder_hierarchy', 'sitegroup', 'mincreated',
 'maxupdated','savets', 'studyenvsitenumber', 'subjectid', 'studysiteid', 'iecmrpar','iecmrpar_cv','cb_crit_deviation_cmt',
 'cb_crit_free_na_cmt','is_unscheduled','sched_dt','tolerance_post','tolerance_pre'],errors='ignore',axis=1)
    return df

In [13]:
def drop_fulfilled_crit_records(df):
    if 'sponsor_crit_result' in df.columns:
        index = df[df.sponsor_crit_result == 'fulfilled'].index
        df = df.drop(index,axis=0)
    return df

In [14]:
def increment_initials(groupby_df):
    counter = 1
    for i,row in groupby_df.iterrows():
        groupby_df.loc[i,'initials'] = row.initials+str(counter)
        counter += 1
    return groupby_df

In [15]:
def str_to_int(list_vals):
    for i in range(len(list_vals)):
        list_vals[i] = int(list_vals[i])
    return list_vals

In [16]:
def reformat_date(date):
    date = date.split(' ')[0].split('.')
    date = date[2]+date[1]+date[0]
    return date

In [17]:
def run_all_for_sf(df):
    df = lowercase_all_vars(df)
    df = drop_cols_for_sf(df)
    df = rename_cols_for_sf(df)
    df = subset_sf_for_sf(df)
    df = df.replace({'in_or_ex':{'Inclusion Criteria':'IN','Inclusion':'IN','Exclusion Criteria':'EX','Exclusion':'EX'}})
#     df = drop_fulfilled_crit_records(df)
    return df

In [18]:
e_224_001 = e_224_001.drop(['Unnamed: 0'],axis=1)
e_224_001.loc[140] = ['100-144','T-M','29/10/2019','SAD Cohort 3 Food Effect Sentinels','Excluded',0,'Inclusion criteria #8-abnormal 12-lead ECG: Possible LVH on ECG; GP letter is sent to the participants']
e_224_001 = insert_row(e_224_001,141,['100-145','EKY','29/10/2019','SAD Cohort 3 Food Effect Sentinels','Eligible - Confirmed for Admission',0,''])
e_224_001.loc[178] = ['100-182','RAH','12/11/2019','SAD Cohort 3 Food Effect ROC 1','Excluded',0,'Excluded due to structural abnormalities detected in Screening Bioclinica ECHO report - Mild MR degree (Exclusion Criteria #2)']
e_224_001 = insert_row(e_224_001,180,['100-183','M-P','12/11/2019','SAD Cohort 3 Food Effect ROC 1','Eligible - Withdrew Consent',0,''])
e_224_001 = insert_row(e_224_001,181,['100-184','TLM','12/11/2019','SAD Cohort 3 Food Effect ROC 2','Excluded',0,'Exclusion # 8. Took multivitamin on 09 NOV 2019.'])
e_224_001.loc[191] = ['100-194','NMC','25/01/2020','SAD Cohort 3 ROC 1','Excluded',0,'Inclusion #11: Participant has poor veins. Unable to cannulate']
e_224_001 = insert_row(e_224_001,192,['100-195','I-A','25/01/2020','SAD Cohort 3 ROC 1','Excluded - Withdrew Consent',0,'Exclusion Criteria #10: Participant mentioned post-screening that she remembered that she had a hole in the heart during childhood and withdrew consent as she does not want to risk being on a Heart study'])
e_224_001['Screening Status'] = e_224_001['Screening Status'].apply(lambda x: x.strip())
e_224_001 = e_224_001.replace({'Screening Status': {'Excluded -':'Excluded','Excluded - Withdrew Consent':'Excluded','Withdrawn Consent - Screening Incomplete':'Excluded','Eligible - Withdrew Consent':'Excluded','Eligible -':'Eligible'}})

C:\Users\dgao\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
e_224_001['ieyn'] = np.where(e_224_001['Screening Status']=='Excluded','N','Y')
e1_461_012['ieyn'] = np.where(e1_461_012.ie_status=='Excluded','N','Y')
e1_461_015['ieyn'] = np.where(e1_461_015.Status=='Screen Fail','N','Y')
e1_461_018['ieyn'] = np.where(e1_461_018.IE_STATUS=='Excluded','N','Y')

In [20]:
e1_461_018 = e1_461_018.drop_duplicates('ID_PARTICIPATION')
e2_461_018 = e2_461_018[['ID_PARTICIPATION','PART_STATE_NAME', 'FIRST_MEDICATION',
'LAST_MEDICATION', 'SCREENING_DT', 'EXCLUSION_REMARK', 'GROUP_NAME', 'STUDY_NO_SPONSOR']]
e_461_018 = pd.merge(e1_461_018,e2_461_018,how='left',on='ID_PARTICIPATION')
e_461_018 = e_461_018[['ID_PARTICIPATION','PERIOD_NAME',
 'IE_STATUS','INC_EXC', 'SPONSOR_NAME', 'PROTOCOL_DESC', 'SPONSOR_CRIT_RESULT',
 'PROTOCOL_PENDING','CB_CRIT_VALUE_TEXT','CB_CRIT_FULFILLED', 'ieyn', 'PART_STATE_NAME', 
 'FIRST_MEDICATION','LAST_MEDICATION', 'SCREENING_DT', 'EXCLUSION_REMARK', 'GROUP_NAME']]

In [21]:
# To add to PATIENT_VISITS table
def create_visit_461_018(df):
    visit_461_018 = pd.DataFrame(columns=['study_id','patient_id','site_id','status','visit_date','reason_for_visit'])
    for i,row in df.iterrows():
        screen_row = ['myk_461_018']
        sf_row = ['myk_461_018']
        random_row = ['myk_461_018']
        screen_row.append(row.ID_PARTICIPATION)
        sf_row.append(row.ID_PARTICIPATION)
        random_row.append(row.ID_PARTICIPATION)
        screen_row.append(row.ID_PARTICIPATION)
        sf_row.append(row.ID_PARTICIPATION)
        random_row.append(row.ID_PARTICIPATION)
        if row.PART_STATE_NAME == 'excluded' or row.PART_STATE_NAME == 'withdrawn':
            screen_row.append('Screen Failed')
            screen_row.append(row.SCREENING_DT)
            screen_row.append('Screening')
            sf_row.append('Screen Failed')
            sf_row.append(row.SCREENING_DT)
            sf_row.append('Screen Fail')
            visit_461_018 = visit_461_018.append(pd.Series(sf_row,index=visit_461_018.columns),ignore_index=True)
        elif row.PART_STATE_NAME == 'completed':
            screen_row.append('Study Completed')
            screen_row.append(row.SCREENING_DT)
            screen_row.append('Screening')
            random_row.append('Study Completed')
            random_row.append(row.FIRST_MEDICATION)
            random_row.append('Randomization')
            visit_461_018 = visit_461_018.append(pd.Series(random_row,index=visit_461_018.columns),ignore_index=True)
        visit_461_018 = visit_461_018.append(pd.Series(screen_row,index=visit_461_018.columns),ignore_index=True)
    return visit_461_018

visit_461_018 = create_visit_461_018(e_461_018)
visit_461_018['patient_id'] = visit_461_018.patient_id.apply(lambda x: str(x))
visit_461_018['patient_id'] = visit_461_018.patient_id.apply(lambda x: '461-018-'+x[:3]+'-'+x[3:])
visit_461_018['site_id'] = 1120
visit_461_018['visit_date'] = visit_461_018.visit_date.apply(lambda x: x[:10].split('.')[::-1])
visit_461_018['visit_date'] = visit_461_018.visit_date.apply(lambda x: pd.to_datetime(''.join(x),format='%Y%m%d'))
PATIENT_VISITS = pd.concat([PATIENT_VISITS,visit_461_018],axis=0)
PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_018']

,visit_id,orig_visit_idx,study_id,patient_id,site_id,status,visit_date,reason_for_visit,assigned_drug,drug_desc,treatment,finished,quant_dispensed
0,NaN,NaN,myk_461_018,461-018-100-830,1120,Study Completed,2020-02-14,Randomization,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,myk_461_018,461-018-100-830,1120,Study Completed,2020-01-17,Screening,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,myk_461_018,461-018-102-024,1120,Study Completed,2020-02-28,Randomization,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,myk_461_018,461-018-102-024,1120,Study Completed,2020-02-17,Screening,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,myk_461_018,461-018-100-495,1120,Screen Failed,2020-01-28,Screen Fail,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,NaN,NaN,myk_461_018,461-018-101-791,1120,Study Completed,2020-02-11,Screening,NaN,NaN,NaN,NaN,NaN
84,NaN,NaN,myk_461_018,461-018-101-962,1120,Study Completed,2020-02-28,Randomization,NaN,NaN,NaN,NaN,NaN
85,NaN,NaN,myk_461_018,461-018-101-962,1120,Study Completed,2020-02-17,Screening,NaN,NaN,NaN,NaN,NaN
86,NaN,NaN,myk_461_018,461-018-100-663,1120,Screen Failed,2020-01-28,Screen Fail,NaN,NaN,NaN,NaN,NaN


In [22]:
elig_224_001 = remove_nan_cols(e_224_001)
elig_461_005 = remove_nan_cols(e1_461_005)
elig_461_006 = remove_nan_cols(e1_461_006)
elig_461_007 = remove_nan_cols(e2_461_007)
elig_461_012 = remove_nan_cols(e1_461_012)
elig_461_015 = remove_nan_cols(e1_461_015)
elig_461_018 = remove_nan_cols(e_461_018)

In [23]:
# Some patients have more than one eligibility record per study, which usually corresponds to a separate row for each different reason for screen fail
elig_224_001 = run_all_for_sf(elig_224_001)
elig_461_005 = run_all_for_sf(elig_461_005)
elig_461_006 = run_all_for_sf(elig_461_006)
elig_461_007 = run_all_for_sf(elig_461_007)
# elig_461_008 = run_all_for_sf(elig_461_008) -- there are ZERO ineligible screened patients
elig_461_012 = run_all_for_sf(elig_461_012)
elig_461_015 = run_all_for_sf(elig_461_015)
elig_461_018 = run_all_for_sf(elig_461_018)

In [24]:
elig_224_001['study_id'] = 'myk_224_001'
elig_461_005['study_id'] = 'myk_461_005'
elig_461_006['study_id'] = 'myk_461_006'
elig_461_007['study_id'] = 'myk_461_007'
elig_461_012['study_id'] = 'myk_461_012'
elig_461_015['study_id'] = 'myk_461_015'
elig_461_018['study_id'] = 'myk_461_018'

In [25]:
def make_in_ex_code_col(df):
    df['temp'] = df['exclusion reason'].apply(lambda x: x.replace(" ","").split('#'))
    df['in_ex_code'] = df.temp.apply(lambda x: int(x[1][0]) if len(x)>1 else x[0])
    for i,row in df.iterrows():
        if row['exclusion reason'] == '':
            df.loc[i,'in_ex_code'] = 0
        elif type(row.in_ex_code) == str:
            for char in row.in_ex_code:
                if char.isdigit():
                    df.loc[i,'in_ex_code'] = int(char)
        if type(row.in_ex_code) == str:
            df.loc[i,'in_ex_code'] = 0
    df = df.drop(['temp'],axis=1)
    return df

In [26]:
elig_224_001['exclusion reason'] = elig_224_001['exclusion reason'].apply(lambda x: x.lower())
elig_224_001['in_or_ex'] = np.where(elig_224_001['exclusion reason'].str.contains('exclusion'),'EX',elig_224_001['exclusion reason'])
elig_224_001['in_or_ex'] = np.where(elig_224_001['exclusion reason'].str.contains('exclsuion'),'EX',elig_224_001['in_or_ex'])
elig_224_001['in_or_ex'] = np.where(elig_224_001['exclusion reason'].str.contains('inclusion'),'IN',elig_224_001['in_or_ex'])
elig_224_001['in_or_ex'] = np.where(elig_224_001['exclusion reason'].str.contains('inclsuion'),'IN',elig_224_001['in_or_ex'])
elig_224_001 = make_in_ex_code_col(elig_224_001)

In [27]:
elig_224_001 = elig_224_001.rename(columns={'exclusion reason':'reason_for_sf','screening no.':'patient_id'})
# elig_224_001 = elig_224_001.groupby('initials').apply(increment_initials)
elig_224_001['patient_id'] = elig_224_001.patient_id.apply(lambda x: '224-001-'+x)
elig_224_001['site_id'] = 2241

In [28]:
def reformat_date_224(date):
    date = date.split('/')
    date = date[2]+date[1]+date[0]
    return date

elig_224_001['assess_date'] = elig_224_001.assess_date.apply(reformat_date_224)

In [29]:
elig_461_006 = elig_461_006.rename(columns={'tivers_std':'protocol_num'})

In [30]:
elig_461_007 = elig_461_007.rename(columns={'iechrt':'cohort'})

In [31]:
elig_461_012['random_no'] = elig_461_012.random_no.apply(lambda x: str(x))
elig_461_012['patient_id'] = elig_461_012.random_no.apply(lambda x: '461-012-1120-'+x[1:])
elig_461_012 = elig_461_012.drop(['random_no'],axis=1)
elig_461_012 = elig_461_012[['study_id','patient_id','assess_date','in_or_ex','in_ex_code','protocol_desc','cb_crit_description','recruit_type']]
elig_461_012 = elig_461_012.rename(columns={'protocol_desc':'reason_for_sf','cb_crit_description':'reason_for_sf2'})
elig_461_012['assess_date'] = elig_461_012.assess_date.apply(reformat_date)
elig_461_012['site_id'] = 1120

In [32]:
elig_461_014 = log_461_014[log_461_014['screen status']=='Excluded'][['unnamed: 1','sid','reason for exclusion','comments','media']].rename(columns={'unnamed: 1':'assess_date','sid':'patient_id','reason for exclusion':'reason_for_sf','comments':'reason_for_sf2','media':'recruit_type'})
elig_461_014['site_id'] = 1120
elig_461_014['patient_id'] = elig_461_014.patient_id.apply(lambda x: '461-014-'+str(int(x)))
elig_461_014['study_id'] = 'myk_461_014'

In [33]:
def convert_site_code_015(site_code):
    site_code = int(site_code)
    if site_code==101:
        site_code = 1151
    if site_code==102:
        site_code = 1152
    return site_code

elig_461_015.columns
elig_461_015['patient_id'] = elig_461_015.patient_id.apply(lambda x: '461-015-115'+x[2:])
elig_461_015 = elig_461_015[['site','study_id','patient_id','date screened','comments (add sf reason if applicable)']]
elig_461_015 = elig_461_015.rename(columns={'site':'site_id','comments (add sf reason if applicable)':'reason_for_sf','date screened':'assess_date'})
elig_461_015['site_id'] = elig_461_015.site_id.apply(lambda x: x[-3:])
elig_461_015['site_id'] = elig_461_015.site_id.apply(convert_site_code_015)

In [34]:
elig_461_018['patient_id'] = elig_461_018.patient_id.apply(lambda x: str(x))
elig_461_018['patient_id'] = elig_461_018.patient_id.apply(lambda x: '461-018-'+x[:3]+'-'+x[3:])
elig_461_018 = elig_461_018[['study_id','patient_id','in_or_ex','in_ex_code','protocol_desc','exclusion_remark']]
elig_461_018['site_id'] = 1120
elig_461_018 = elig_461_018.rename(columns={'protocol_desc':'reason_for_sf','exclusion_remark':'reason_for_sf2'})

In [35]:
# DEAN CAN ADD SCREEN FAILS FROM 014 BACK TO SCREEN_FAILS

SCREEN_FAILS = pd.concat([elig_224_001,elig_461_005,elig_461_006,elig_461_007,elig_461_012,elig_461_014,elig_461_015,elig_461_018],axis=0)
SCREEN_FAILS = pd.concat([SCREEN_FAILS,missed_sfs],join='outer')
SCREEN_FAILS = SCREEN_FAILS.reset_index(drop=True)
SCREEN_FAILS['in_ex_code'] = SCREEN_FAILS.in_ex_code.apply(format_in_ex_code)
SCREEN_FAILS = SCREEN_FAILS.replace({'in_ex_code':{'No Data':0}})
SCREEN_FAILS['in_ex_code'] = SCREEN_FAILS.in_ex_code.apply(lambda x: int(x))
SCREEN_FAILS = SCREEN_FAILS.groupby('patient_id').apply(merge_in_ex_codes)
SCREEN_FAILS['in_id'] = SCREEN_FAILS.in_id.apply(lambda x: x.split(',')[:-1])
SCREEN_FAILS['ex_id'] = SCREEN_FAILS.ex_id.apply(lambda x: x.split(',')[:-1])
SCREEN_FAILS['in_id'] = SCREEN_FAILS.in_id.apply(str_to_int)
SCREEN_FAILS['ex_id'] = SCREEN_FAILS.ex_id.apply(str_to_int)
SCREEN_FAILS['assess_date'] = SCREEN_FAILS.assess_date.apply(lambda x: pd.to_datetime(x,format='%Y%m%d') if type(x) == str else x)
SCREEN_FAILS1 = SCREEN_FAILS[['study_id','site_id','patient_id','recruit_type','cohort','protocol_num','assess_date','in_id','ex_id','reason_for_sf','reason_for_sf2','ierscrn_cv']]

In [36]:
def fill_004_study_id(groupby_df):
    for i,row in groupby_df.iterrows():
        if row.study_id == 'myk_461_004':
            groupby_df.loc[i,'site_id'] += int(row.patient_id[8:11])
        else:
            groupby_df.loc[i,'site_id'] = groupby_df.loc[i,'site_id']
    return groupby_df

In [37]:
SCREEN_FAILS1 = SCREEN_FAILS1.drop_duplicates('patient_id')
SCREEN_FAILS1['site_id'] = SCREEN_FAILS1.site_id.fillna(0)
SCREEN_FAILS1['site_id'] = SCREEN_FAILS1.site_id.apply(lambda x: int(x))

In [38]:
sf_visits = {}
for each in PATIENT_VISITS.columns[:-5]:
    sf_visits[each] = []

# Finds every row from PATIENT_VISITS where the patient_id matches a patient_id in SCREEN_FAILS1
# This is to double check that patients in SCREEN_FAILS do not appear in PATIENT_VISITS as 'randomized'
# IF they do, it drops them from SCREEN_FAILS
for i,row in PATIENT_VISITS.iterrows():
    if row['patient_id'] in SCREEN_FAILS1.patient_id.values:
        sf_visits['visit_id'].append(row.visit_id)
        sf_visits['orig_visit_idx'].append(row.orig_visit_idx)
        sf_visits['study_id'].append(row.study_id)
        sf_visits['patient_id'].append(row.patient_id)
        sf_visits['site_id'].append(row.site_id)
        sf_visits['status'].append(row.status)
        sf_visits['visit_date'].append(row.visit_date)
        sf_visits['reason_for_visit'].append(row.reason_for_visit)  

In [39]:
sf_visits = pd.DataFrame(sf_visits,columns=[col for col in PATIENT_VISITS.columns[:-5].values])
ids_to_drop = sf_visits[sf_visits.reason_for_visit=='Randomization'][['patient_id','study_id']].values
ids_to_drop = ids_to_drop.tolist()

In [40]:
print("SF before check: ",SCREEN_FAILS1.shape)
for i,row in SCREEN_FAILS1.iterrows():
    to_check = [row['patient_id'],row['study_id']]
    if to_check in list(ids_to_drop):
        SCREEN_FAILS1 = SCREEN_FAILS1.drop(i)

print("SF after check/dropping 'randomized' patients: ",SCREEN_FAILS1.shape)

SF before check:  (709, 12)
SF after check/dropping 'randomized' patients:  (677, 12)


In [41]:
SCREEN_FAILS1 = SCREEN_FAILS1.reset_index()
sf_in_visits = set(PATIENT_VISITS.patient_id.values) & set(SCREEN_FAILS1.patient_id.values)
sf_not_in_visits = set(SCREEN_FAILS1.patient_id.values) - sf_in_visits
sf_to_add_idx = []
for i,row in SCREEN_FAILS1.iterrows():
    if row.patient_id in sf_not_in_visits:
        sf_to_add_idx.append(i)
SCREEN_FAILS1 = SCREEN_FAILS1.drop(['index'],axis=1)
sf_add_to_visits = SCREEN_FAILS1.loc[sf_to_add_idx,['study_id','site_id','patient_id','assess_date']]
sf_add_to_visits['status'] = 'Screen Failed'
sf_add_to_visits['reason_for_visit'] = 'Screen Fail'
sf_add_to_visits = sf_add_to_visits.rename(columns={'assess_date':'visit_date'})

print('Number of patient IDs to add to visits: ',len(sf_to_add_idx))
print('Num of rows in visits BEFORE rows added from SF: ',PATIENT_VISITS.shape[0])
PATIENT_VISITS = pd.concat([PATIENT_VISITS,sf_add_to_visits],axis=0)
print('This is the new # of rows in visits (should be 305 more)',PATIENT_VISITS.shape[0])

Number of patient IDs to add to visits:  290
Num of rows in visits BEFORE rows added from SF:  9366
This is the new # of rows in visits (should be 305 more) 9656


In [42]:
PATIENT_VISITS.loc[(PATIENT_VISITS.study_id=='myk_224_001') & (PATIENT_VISITS.status=='Screen Failed'),'reason_for_visit'] = "Screen Fail"

In [43]:
# DO NOT USE BECAUSE THESE FILES FOR 014 and 018 WERE INCORRECT FILES TO USE FOR PATIENT_VISITS
# INSTEAD, OTHER FILES HAVE BEEN USED
# 014 was added to PATIENT_VISITS in the original patient_visits file
# 018 was added from this file

def convert_018_to_datetime(str_date):
    date = str_date[:10]
    date = date.split('.')[::-1]
    date = ''.join(date)
    date = pd.to_datetime(date,format='%Y%m%d')
    return date

# Adding all the screening patients from SF table into visits table
# visit_461_018 = e2_461_018[['STUDY_NO_SPONSOR','ID_PARTICIPATION','PART_STATE_NAME','SCREENING_DT']]
# visit_461_018 = visit_461_018.rename(columns={'STUDY_NO_SPONSOR':'study_id','ID_PARTICIPATION':'patient_id','PART_STATE_NAME':'status','SCREENING_DT':'visit_date'})
# visit_461_018 = visit_461_018.replace({'completed':'Treatment Completed','excluded':'Screen Failed','withdrawn':'Early Terminated'})
# visit_461_018['patient_id'] = visit_461_018.patient_id.apply(lambda x: str(x))
# visit_461_018['patient_id'] = visit_461_018.patient_id.apply(lambda x: '461-018-'+x[:3]+'-'+x[3:])
# visit_461_018['study_id'] = 'myk_461_018'
# visit_461_018['visit_date'] = visit_461_018.visit_date.apply(convert_018_to_datetime)
# visit_461_018['reason_for_visit'] = 'Screening'
# visit_461_018['site_id'] = 1120

# Adding all the screening patients from SF table into visits table
# visit_461_014 = e1_461_014[e1_461_014.ieyn=='Y']
# visit_461_014 = visit_461_014[['id_participation','period_name','actual_dt']]
# visit_461_014 = visit_461_014.rename(columns={'id_participation':'patient_id','period_name':'reason_for_visit','actual_dt':'visit_date'})
# visit_461_014['patient_id'] = visit_461_014.patient_id.apply(lambda x: str(x))
# visit_461_014['patient_id'] = visit_461_014.patient_id.apply(lambda x: '461-014-'+x)
# visit_461_014['study_id'] = 'myk_461_014'
# visit_461_014 = visit_461_014.replace({'SCR':'Screening','P01':'Day 1'})
# visit_461_014['visit_date'] = visit_461_014.visit_date.apply(convert_018_to_datetime)
# visit_461_014['status'] = 'Randomized'
# visit_461_014['site_id'] = 1120
# visit_461_014 = visit_461_014.drop_duplicates(['reason_for_visit','patient_id'])

In [44]:
PATIENT_VISITS = PATIENT_VISITS.drop('visit_id',axis=1)
PATIENT_VISITS = PATIENT_VISITS.groupby(['patient_id','study_id']).apply(fill_status).reset_index(drop=True)

In [45]:
# Input error on randomization date for patient_id == 461-005-302-504
# This line corrects the date from 2020 to 2019
PATIENT_VISITS.loc[4476,'visit_date'] = pd.to_datetime('20190405',format='%Y%m%d')

In [46]:
PATIENTS_FOR_MOD = PATIENT_VISITS.replace({'myk_491_003_mad':'myk_491_003','myk_491_003_sad':'myk_491_003','Study Completed':'Randomized','Screened':'Randomized'})

In [47]:
PATIENT_VISITS.to_csv('C:/Users/dgao/Desktop/Spotfire_DATA/all_visits.csv',date_format='%Y%m%d')
SCREEN_FAILS1.to_csv('C:/Users/dgao/Desktop/Spotfire_DATA/all_sf_patients.csv')
PATIENTS_FOR_MOD.to_csv('C:/Users/dgao/Desktop/Spotfire_DATA/patients_for_mod.csv',date_format='%Y%m%d')

In [48]:
sites_005 = PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_005']
sites_005.to_csv('C:/Users/dgao/Desktop/Spotfire_DATA/all_visit_461_005.csv')

In [49]:
sites_006 = PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_006']
sites_006.to_csv('C:/Users/dgao/Desktop/Spotfire_DATA/to_test_visit_461_006.csv')

In [50]:
PATIENT_VISITS[(PATIENT_VISITS.study_id=='myk_224_001')&(PATIENT_VISITS.status=='Screen Failed')]
PATIENT_VISITS[PATIENT_VISITS.patient_id=='461-005-998-506']
# PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_224_001']
PATIENT_VISITS.head(60)
PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_015']
PATIENT_VISITS[(PATIENT_VISITS.reason_for_visit=='Screen Fail')&(PATIENT_VISITS.study_id=='myk_461_005')].sort_values('visit_date')
# PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_005'].reason_for_visit.value_counts()
PATIENT_VISITS[(PATIENT_VISITS.reason_for_visit=='Randomization')&(PATIENT_VISITS.study_id=='myk_461_005')].sort_values('visit_date')
PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_005'].sort_values(['patient_id','visit_date']).head(60)
PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_012']

,orig_visit_idx,study_id,patient_id,site_id,status,visit_date,reason_for_visit,assigned_drug,drug_desc,treatment,finished,quant_dispensed
7929,0.0,myk_461_012,461-012-1120-001,1120,Randomized,2018-11-21 13:16:00,SCREENING,,,,,
7930,1.0,myk_461_012,461-012-1120-001,1120,Randomized,2018-12-02 10:26:00,PERIOD 1 DAY -1,,,,,
7931,2.0,myk_461_012,461-012-1120-001,1120,Randomized,2018-12-03 20:00:00,PERIOD 1 DAY 1,,,,,
7932,3.0,myk_461_012,461-012-1120-001,1120,Randomized,2018-12-04 10:08:00,PERIOD 1 DAY 2,,,,,
7933,4.0,myk_461_012,461-012-1120-001,1120,Randomized,2018-12-05 08:41:00,PERIOD 1 DAY 3,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
8114,183.0,myk_461_012,461-012-1120-015,1120,Randomized,2019-02-11 07:41:00,PERIOD 1 DAY 35,,,,,
8115,184.0,myk_461_012,461-012-1120-015,1120,Randomized,2019-02-21 08:42:00,PERIOD 1 DAY 45,,,,,
8116,185.0,myk_461_012,461-012-1120-015,1120,Randomized,2019-03-08 09:11:00,FOLLOW-UP,,,,,
8117,1.0,myk_461_012,461-012-1120-016,1120,Randomized,2019-04-01 00:00:00,Screening,,,,,


DATA QUALITY CHECK

In [51]:
PATIENT_VISITS[(PATIENT_VISITS.study_id=='myk_224_001') & (PATIENT_VISITS.reason_for_visit=='Screen Fail')]
# PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_224_001'].reason_for_visit.value_counts()
PATIENT_VISITS[(PATIENT_VISITS.study_id=='myk_224_001') & PATIENT_VISITS.orig_visit_idx.isna()]
ids_from_007_pvs = set(PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_007'].patient_id.values)
# ids_from_007_patients = set(PATIENTS[PATIENTS.patient_id.str.contains('461-007')].patient_id.values)
# missing_from_007_pvs = set(ids_from_007_patients) - set(ids_from_007_pvs)
# missing_from_007_pvs
# PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_012'].site_id.value_counts()

In [52]:
PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_005'].status.value_counts()
PATIENT_VISITS[(PATIENT_VISITS.study_id=='myk_461_005')&(PATIENT_VISITS.status=='Prescreen Failed')].query('reason_for_visit=="Screening"')
PATIENT_VISITS[(PATIENT_VISITS.study_id=='myk_461_005')&(PATIENT_VISITS.status=='Prescreen Failed')].reason_for_visit.value_counts()

# DEAN -- the patient_ids below are examples of WHY this was coded as 'Prescreen Fail' (but should it instead be coded as screen fail?)
# I ended up recoding TEN patient records from prescreen fail to screen fail BY writing a condition to catch study 461-005 in the fill_status function
PATIENT_VISITS[PATIENT_VISITS.patient_id=='461-005-102-503'].sort_values('visit_date')
PATIENT_VISITS[PATIENT_VISITS.patient_id=='461-005-407-506'].sort_values('visit_date')
# PATIENT_VISITS[PATIENT_VISITS.patient_id=='461-005-799-502'].sort_values('visit_date')
PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_012'].patient_id.value_counts()

461-012-1120-009    16
461-012-1120-001    15
461-012-1120-011    14
461-012-1120-012    14
461-012-1120-010    13
461-012-1120-013    13
461-012-1120-002    13
461-012-1120-015    13
461-012-1120-004    13
461-012-1120-006    13
461-012-1120-014    13
461-012-1120-005    13
461-012-1120-007    12
461-012-1120-003    11
461-012-1120-008     2
461-012-1120-016     2
Name: patient_id, dtype: int64

In [53]:
len(PATIENT_VISITS[(PATIENT_VISITS.study_id=='myk_461_007')&(PATIENT_VISITS.status=='Screen Failed')].patient_id.value_counts())
# PATIENT_VISITS[PATIENT_VISITS.patient_id=='461-005-102-503'].sort_values('visit_date')
# PATIENT_VISITS[PATIENT_VISITS.patient_id=='461-014-91819']
# PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_005']

18

In [54]:
t = PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_014']
len(t.patient_id.value_counts())
t[t.status=='Screen Failed']

,orig_visit_idx,study_id,patient_id,site_id,status,visit_date,reason_for_visit,assigned_drug,drug_desc,treatment,finished,quant_dispensed
8334,40.0,myk_461_014,461-014-45374,1120,Screen Failed,2019-07-22,Screening,,,,,
8345,16.0,myk_461_014,461-014-70518,1120,Screen Failed,2019-07-08,Screening,,,,,
8348,4.0,myk_461_014,461-014-73670,1120,Screen Failed,2019-06-18,Screening,,,,,
8359,31.0,myk_461_014,461-014-78298,1120,Screen Failed,2019-07-18,Screening,,,,,
8360,6.0,myk_461_014,461-014-80420,1120,Screen Failed,2019-06-18,Screening,,,,,
8361,14.0,myk_461_014,461-014-80514,1120,Screen Failed,2019-07-05,Screening,,,,,
8362,2.0,myk_461_014,461-014-80958,1120,Screen Failed,2019-06-14,Screening,,,,,
8363,3.0,myk_461_014,461-014-81257,1120,Screen Failed,2019-06-17,Screening,,,,,
8371,27.0,myk_461_014,461-014-82661,1120,Screen Failed,2019-07-10,Screening,,,,,
8372,43.0,myk_461_014,461-014-83656,1120,Screen Failed,2019-07-23,Screening,,,,,


In [55]:
SCREEN_FAILS1[SCREEN_FAILS1.recruit_type.notna()]
PATIENT_VISITS

,orig_visit_idx,study_id,patient_id,site_id,status,visit_date,reason_for_visit,assigned_drug,drug_desc,treatment,finished,quant_dispensed
0,NaN,myk_224_001,224-001-100-001,2241,Screen Failed,2019-07-25,Screen Fail,NaN,NaN,NaN,NaN,NaN
1,NaN,myk_224_001,224-001-100-002,2241,Screen Failed,2019-07-25,Screen Fail,NaN,NaN,NaN,NaN,NaN
2,NaN,myk_224_001,224-001-100-003,2241,Screen Failed,2019-07-26,Screen Fail,NaN,NaN,NaN,NaN,NaN
3,NaN,myk_224_001,224-001-100-004,2241,Screen Failed,2019-07-26,Screen Fail,NaN,NaN,NaN,NaN,NaN
4,NaN,myk_224_001,224-001-100-005,2241,Screen Failed,2019-07-26,Screen Fail,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9651,405.0,myk_491_003_mad,491-003-401-101,491401,Randomized,2019-05-23,D9,,,,,
9652,406.0,myk_491_003_mad,491-003-401-101,491401,Randomized,2019-05-24,D10,,,,,
9653,407.0,myk_491_003_mad,491-003-401-101,491401,Randomized,2019-05-25,D11,,,,,
9654,408.0,myk_491_003_mad,491-003-401-101,491401,Randomized,2019-05-26,D12,,,,,


In [56]:
PATIENTS_FOR_MOD[PATIENTS_FOR_MOD.site_id==102]
# PATIENT_VISITS

,orig_visit_idx,study_id,patient_id,site_id,status,visit_date,reason_for_visit,assigned_drug,drug_desc,treatment,finished,quant_dispensed
671,61.0,myk_461_004,461-004-102-011,102,Randomized,2016-10-21,Screening,,,,,
672,62.0,myk_461_004,461-004-102-011,102,Randomized,2016-10-28,Week 1,,,,,
673,63.0,myk_461_004,461-004-102-011,102,Randomized,2016-10-31,Week 1,,,,,
674,64.0,myk_461_004,461-004-102-011,102,Randomized,2016-11-04,Week 1,,,,,
675,65.0,myk_461_004,461-004-102-011,102,Randomized,2016-11-11,Week 2,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
9091,12.0,myk_491_003,491-003-102-103,102,Randomized,2019-05-22,D10,,,,,
9092,13.0,myk_491_003,491-003-102-103,102,Randomized,2019-05-23,D11,,,,,
9093,14.0,myk_491_003,491-003-102-103,102,Randomized,2019-05-24,D12,,,,,
9094,15.0,myk_491_003,491-003-102-103,102,Randomized,2019-05-30,FU,,,,,


In [59]:
PATIENTS_FOR_MOD.patient_id.value_counts()
SCREEN_FAILS1.study_id.value_counts()

myk_224_001        262
myk_461_005        177
myk_461_006         98
myk_491_003_mad     33
myk_461_015         28
myk_461_007         25
myk_491_003_sad     19
myk_461_014         16
myk_461_018         14
myk_461_004          4
myk_461_012          1
Name: study_id, dtype: int64

In [62]:
e_224_001['Exclusion Reason'].values

array(['Withdrew consent -  ',
       'Inclusion criteria #8- abnormal ECG, QRS above 110', '  ',
       'Exclusion criteria #3 - cardicac abnormality, excluded due to new cardiac murmur',
       'Withdrew from study as now living in Perth ',
       'Inclusion critiera #5 - LVEF less than 55% ',
       'Inclusion criteria #8 - abnormal ECG, PR interval below range (118)',
       'Exclusion criteria #10 - medical history, significant bowel surgery',
       '  ', 'Exclusion criteria #6 - evidence of active infection',
       'Exclusion criteria #10 - medical history, anxiety',
       'Withdrew consent - unable to commit to study dates due to other commitments',
       'Exclusion criteria # 3 - excluded due to possible AF on ECG',
       '  ',
       'Exclusion criteria 6 - active infection, active tinea vesicolour',
       'Exclusion #2 - abnormal TTE ',
       'Exclusion criteria #7 - positive DOA, THC ',
       'Exclusion criteria #3 - cardiac abnormality, multiple premature ventricula

In [86]:
PATIENTS_FOR_MOD[PATIENTS_FOR_MOD.visit_date.notna()].sort_values('visit_date')

,orig_visit_idx,study_id,patient_id,site_id,status,visit_date,reason_for_visit,assigned_drug,drug_desc,treatment,finished,quant_dispensed
696,86.0,myk_461_004,461-004-102-013,102,Randomized,2016-10-19,Screening,,,,,
671,61.0,myk_461_004,461-004-102-011,102,Randomized,2016-10-21,Screening,,,,,
806,0.0,myk_461_004,461-004-103-021,103,Randomized,2016-10-26,Screening,,,,,
687,77.0,myk_461_004,461-004-102-012,102,Early Terminated,2016-10-26,Screening,,,,,
697,99.0,myk_461_004,461-004-102-013,102,Randomized,2016-10-26,Unscheduled,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
518,280.0,myk_224_001,224-001-100-303,2241,Randomized,2020-07-07,Week 1,,,,,
552,297.0,myk_224_001,224-001-100-316,2241,Randomized,2020-07-07,Week 1,,,,,
526,272.0,myk_224_001,224-001-100-304,2241,Randomized,2020-07-07,Week 1,,,,,
567,NaN,myk_224_001,224-001-100-329,2241,Screen Failed,2020-07-21,Screen Fail,NaN,NaN,NaN,NaN,NaN
